## intro :: generic predefined section kgap-demo

### defines standard scripts backed by `sema.query`

In [1]:
from kgap_tools import execute_to_df, generate_sparql

### show the generate_sparql expansion from the templates in `./queries/*.sparql`

In [2]:
print("generated sparql in action...")
sparql: str = generate_sparql("all.sparql", N=5) # general spo query with limit 5
print(sparql)

generated sparql in action...
SELECT * WHERE { ?s ?p ?o. }
LIMIT 5


### just execute that query to pandas dataframe

In [3]:
execute_to_df("all.sparql", N=5)

,s,p,o
0,genid-3ef869bc5c4946f38793fefa95da3c3c8-b0_df_...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://aphia.org/ns/taxon-name/Taxon
1,genid-3ef869bc5c4946f38793fefa95da3c3c8-b0_df_...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2004/02/skos/core#Concept
2,https://aphia.org/id/taxname/1864475,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://aphia.org/ns/taxon-name/TaxonName
3,https://aphia.org/id/taxname/1864475,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://www.w3.org/2008/05/skos-xl#Label
4,genid-3ef869bc5c4946f38793fefa95da3c3c8-b0_df_...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://aphia.org/ns/taxon-name/Taxon


# Aphia.org SYNC DASHBOARD

In [4]:
from kgap_tools import execute_to_df, generate_sparql
params: dict = dict()

## find types

In [10]:
params = dict(regex="aphia") 
execute_to_df("rdf-types-count.sparql", **params)

,type,count
0,https://aphia.org/ns/taxon-name/Taxon,450000
1,https://aphia.org/ns/taxon-name/TaxonName,225000


## find taxname subjects

In [8]:
params = dict()
execute_to_df("aphia-taxnames.sparql", **params)

,tn
0,https://aphia.org/id/taxname/1864475
1,https://aphia.org/id/taxname/1864476
2,https://aphia.org/id/taxname/1864488
3,https://aphia.org/id/taxname/1864490
4,https://aphia.org/id/taxname/1864489
...,...
153995,https://aphia.org/id/taxname/1781500
153996,https://aphia.org/id/taxname/1781508
153997,https://aphia.org/id/taxname/1781544
153998,https://aphia.org/id/taxname/1781547


## find predicates from aphia_id

In [9]:
aphiaids: list[int] = [] # use empty list to get them all -- alternative, make a pick: [1864475, 1864476, ]
preds: list[str] = []
params = dict(aphiaids=aphiaids, preds=preds)
execute_to_df("aphia-taxn-predicates.sparql", **params)

,pred,tn,value,value_ln,value_type
0,https://aphia.org/ns/taxon-name/fullScientific...,https://aphia.org/id/taxname/1864475,"Hypelasma salevensis (Favre, 1913)",,http://www.w3.org/2001/XMLSchema#string
1,https://aphia.org/ns/taxon-name/fullScientific...,https://aphia.org/id/taxname/1864476,"Matheronia salevensis Favre, 1913",,http://www.w3.org/2001/XMLSchema#string
2,https://aphia.org/ns/taxon-name/fullScientific...,https://aphia.org/id/taxname/1864488,"Parabopyrella frevo Modesto, Ribeiro & Souza-F...",,http://www.w3.org/2001/XMLSchema#string
3,https://aphia.org/ns/taxon-name/fullScientific...,https://aphia.org/id/taxname/1864490,"Acarinina saudica Anan, 2025",,http://www.w3.org/2001/XMLSchema#string
4,https://aphia.org/ns/taxon-name/fullScientific...,https://aphia.org/id/taxname/1864489,"Tollmannia qatarica Anan, 2025",,http://www.w3.org/2001/XMLSchema#string
...,...,...,...,...,...
361995,https://rs.tdwg.org/dwc/terms/scientificName,https://aphia.org/id/taxname/1772600,"Rostellaria powisii var. abyssicola Schepman, ...",,http://www.w3.org/2001/XMLSchema#string
361996,https://rs.tdwg.org/dwc/terms/scientificName,https://aphia.org/id/taxname/1772607,"Helicina cataliniana G. B. Sowerby II, 1873",,http://www.w3.org/2001/XMLSchema#string
361997,https://rs.tdwg.org/dwc/terms/scientificName,https://aphia.org/id/taxname/1506453,"Helicina ochracea Poey, 1852",,http://www.w3.org/2001/XMLSchema#string
361998,https://rs.tdwg.org/dwc/terms/scientificName,https://aphia.org/id/taxname/1506510,"Helicina rubrocincta Poey, 1854",,http://www.w3.org/2001/XMLSchema#string


# TODO -- add lookup by name functions
* alternative by tapping into the sparql endpoint
* alternative by calling to the worms-webservice